In [ ]:
%pip install llama-index-llms-openai
!pip install llama-index
!pip install --upgrade llama-index-core llama-index-llms-openai llama-index-utils-workflow


import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.ERROR)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

import nest_asyncio
nest_asyncio.apply()


In [ ]:

from llama_index.core.workflow import (
    StartEvent,
    StopEvent,
    Workflow,
    step,
)

from llama_index.core.workflow import draw_all_possible_flows


In [ ]:

from llama_index.core import Settings
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.ollama import OllamaEmbedding

# Configure Ollama LLM
ollama_llm = Ollama(
    model="llama3.2:latest",
    base_url="http://localhost:11434",
    temperature=0.1
)

# Configure embedding model
ollama_embedding = OllamaEmbedding(
    model_name="nomic-embed-text:latest",
    base_url="http://localhost:11434",
    ollama_additional_kwargs={"mirostat": 0}
)

Settings.llm = ollama_llm
Settings.embed_model = ollama_embedding

In [5]:


# Define the Ollama instance globally or as part of the workflow
class WorkflowGenerator(Workflow):
    def __init__(self, timeout: int, verbose: bool = False):
        super().__init__(timeout=timeout, verbose=verbose)
        self.llm = Ollama(
            base_url="http://localhost:11434",  # Your local Ollama setup
            model="llama3.2:latest",            # The preferred model
            temperature=0.1                     # Controlled temperature for deterministic results
        )

    @step
    async def generate(self, ev: StartEvent) -> StopEvent:
        response = await self.llm.acomplete(ev.query)  # Use the llm defined in the class
        return StopEvent(result=str(response))


In [ ]:

# Initialize and run the workflow
w = WorkflowGenerator(timeout=10, verbose=False)
result = await w.run(query="What's cosine similarity?")
print(result)


In [ ]:
draw_all_possible_flows(WorkflowGenerator, filename="trivial_workflow.html")